In [2]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

### Data Selection

In [3]:
# Path to the celebA id text file
dat_path = "../../../Downloads/identity_CelebA.txt"
imgDf = pd.read_csv(dat_path, sep=' ', header=None, names=['img_name', 'class_id'], )

In [4]:
# Acquires some useful information for dataset creation. K essentially defines how many classes to use in the subproblem.
# In this case, we simply use the classes with the highest number of images so that a small train/val set can be made
K = 10
topKClasses = list(imgDf['class_id'].value_counts().nlargest(K).index)
topKImages = imgDf.loc[imgDf['class_id'].isin(topKClasses)].groupby('class_id')

In [5]:
# Some simple visualization
print(topKClasses)

classDict = dict()
for key, item in topKImages:
    classDict[key] = list(item.values[:,0])
print(classDict)

[3782, 3227, 2820, 3745, 3699, 9152, 8968, 9256, 6568, 1757]
{1757: ['001786.jpg', '009205.jpg', '010068.jpg', '010092.jpg', '015487.jpg', '022885.jpg', '029964.jpg', '037947.jpg', '039638.jpg', '044981.jpg', '049471.jpg', '050173.jpg', '053239.jpg', '053740.jpg', '056513.jpg', '058193.jpg', '065581.jpg', '071786.jpg', '082512.jpg', '086944.jpg', '089511.jpg', '089991.jpg', '096209.jpg', '116292.jpg', '129311.jpg', '129525.jpg', '141053.jpg', '142833.jpg', '149901.jpg', '157739.jpg', '160434.jpg'], 2820: ['001553.jpg', '002558.jpg', '003568.jpg', '005265.jpg', '008235.jpg', '008286.jpg', '012101.jpg', '014715.jpg', '016174.jpg', '018144.jpg', '018909.jpg', '028690.jpg', '034377.jpg', '034769.jpg', '043941.jpg', '045344.jpg', '056764.jpg', '062065.jpg', '067445.jpg', '075590.jpg', '081558.jpg', '085080.jpg', '087813.jpg', '097468.jpg', '114148.jpg', '115262.jpg', '116428.jpg', '130485.jpg', '130858.jpg', '131983.jpg', '137347.jpg', '150905.jpg', '154010.jpg', '156283.jpg', '157826.jpg']

In [10]:
# Separate relevant files
alignedPrefix = '../../../Downloads/img_align_celeba/'
# nonAlignedPrefix = '../../../Downloads/img_celeba/'

# destNonaligned = '../CelebA/SourceUnaligned/'
destAligned = '../CelebA/datasets/'

# create dirs and transfer images
# os.makedirs(destNonaligned, exist_ok=True)
os.makedirs(destAligned, exist_ok=True)

for cName in classDict:
    # create class dir if necessary
#     cnDir = destNonaligned + str(cName) + '/'
    caDir = destAligned + str(cName) + '/'
#     if not os.path.isdir(cnDir):
#         os.mkdir(cnDir)
    if not os.path.isdir(caDir):
        os.mkdir(caDir)
    
    for imageFN in classDict[cName]:
        # first copy the aligned images
        oPath = os.path.join(alignedPrefix, imageFN)
        destPath = os.path.join(caDir, imageFN)
        shutil.copyfile(oPath, destPath)
        
        # and now the unaligned ones
#         oPath = os.path.join(nonAlignedPrefix, imageFN)
#         destPath = os.path.join(cnDir, imageFN)
#         shutil.copyfile(oPath, destPath)

### Data BBox Conversion (YOLO)

JSON Structure:
``` Python
  tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
```

In [ ]:
# Misc needed to set some FLAG for TF
class Object():
    pass
FLAGS = Object()
FLAGS.output_path = '../../YoloTest/yolov3-tf2/data/train_faces.tfrecord'
FLAGS.val_path = '../../YoloTest/yolov3-tf2/data/val_faces.tfrecord'

def create_tf_example(example):
    # load particular image and populate following parts
    img_raw = open(example['img_path'], 'rb').read()
    img_mat = plt.imread(example['img_path'])
    height = img_mat.shape[0] # Image height
    width = img_mat.shape[1] # Image width
    
    # Calculated using values from example
    xmin = [1.0*example['xmin']/width,] # List of normalized left x coordinates in bounding box (1 per box)
    xmax = [1.0*example['xmax']/width,] # List of normalized right x coordinates in bounding box
                         # (1 per box)
    ymin = [1.0*example['ymin']/height,] # List of normalized top y coordinates in bounding box (1 per box)
    ymax = [1.0*example['ymax']/height,] # List of normalized bottom y coordinates in bounding box
                         # (1 per box)
    classes_text = [example['label_text'],] # List of string class name of bounding box (1 per box)
    classes = [example['label'],] # List of integer class id of bounding box (1 per box)
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[
            example['filename'].encode('utf8')])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[
            example['filename'].encode('utf8')])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=['jpeg'.encode('utf8')])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmin)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmax)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymin)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymax)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }))
    return tf_example

''' OUTPUT: SAMPLE_IMG_PATH / FILENAME / X_MIN / X_MAX / Y_MIN / Y_MAX / LABEL_INT / LABEL_TEXT'''
def read_dataset(bboxPath, imsPath):
    # output is list of dicts
    outSamples = []
    
    # Reads in image names line by line
    with open(bboxPath, 'r') as inFile:
        bboxLines = inFile.readlines()
        
    # Remove first two lines
    bboxLines = bboxLines[2:]
    
    # Now process each line individually
    for imageLine in bboxLines:
        example = dict()
        filename, x1, y1, w, h = imageLine.split()
        example['img_path'] = imsPath + filename
        example['filename'] = filename
        example['xmin'] = int(x1)
        example['ymin'] = int(y1)
        example['xmax'] = int(x1) + int(w)
        example['ymax'] = int(y1) + int(h)
        example['label_text'] = "face".encode('utf-8')
        example['label'] = 0
        
        outSamples.append(example)
        
    return outSamples

# read in your dataset to examples variable
examples = read_dataset('../../../Downloads/list_bbox_celeba.txt', '../../Downloads/img_celeba/')
# trainLen = int(0.8*(len(examples)))
trainLen = 20000
valLen = 10000

# Write the dataset files
writer = tf.io.TFRecordWriter(FLAGS.output_path)
for example in tqdm(examples[:trainLen]):
    tf_example = create_tf_example(example)
    writer.write(tf_example.SerializeToString())
writer.close()

# write validation now
writer = tf.io.TFRecordWriter(FLAGS.val_path)
for example in tqdm(examples[trainLen+1:trainLen+valLen+1]):
    tf_example = create_tf_example(example)
    writer.write(tf_example.SerializeToString())
writer.close()